# 🌟 XL LoRA Trainer (Lightning.ai) por Mighty Crimson

Este notebook adapta el flujo de trabajo del proyecto original de [hollowstrawberry](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) y el fork de [whitez](https://colab.research.google.com/github/gwhitez/Lora-Trainer-XL/blob/main/Fix_Lora_Trainer_XL.ipynb) usando [Kohya](https://github.com/kohya-ss/sd-scripts/tree/5a18a03ffcc2a21c6e884a25d041076911a79a2a) a para ejecutarse dentro de Lightning.ai.<br>
Si se les hace compicado entender todo este notebook, comienzen primero haciendo loras en colab para familiarizarse con este notebook.<br>
Recomendable solo usar la grafica L4, ya que es suficiente para todo.<br>
Todo el trabajo se realiza en el directorio base `/teamspace/studios/this_studio`.

> Basado en el trabajo de [Kohya-ss](https://github.com/kohya-ss/sd-scripts), [Linaqruf](https://github.com/Linaqruf/kohya-trainer) y los colaboradores de este fork.


### ⭕ Aviso
Este cuaderno está pensado para investigación y entrenamiento de modelos LoRA de forma responsable.
Asegúrate de respetar los términos de servicio de Lightning.ai y de cualquier repositorio o dataset que utilices.


| Recurso | Enlace |
| :--- | :--- |
| Código base del entrenador | [gwhitez/LoRA_Easy_Training_scripts_Backend](https://github.com/gwhitez/LoRA_Easy_Training_scripts_Backend) |
| Scripts originales | [kohya-ss/sd-scripts](https://github.com/kohya-ss/sd-scripts) |
| Adaptación Lightning | Este notebook |


# Dependencias, solo inicia una vez esta celda y reinicia el kernel

In [ ]:
## Dependencias, solo inicia una vez esta celda y reinicia el kernel
from IPython.display import display, HTML
from IPython.display import clear_output
import shutil
!git clone https://github.com/MightyCrimsonX/LoRA_Easy_Training_scripts_Backend.git
# Instalar cada paquete individualmente
!sudo apt install aria2 -q
%cd /teamspace/studios/this_studio/LoRA_Easy_Training_scripts_Backend
!git submodule init
!git submodule update
!uv pip install -r requirements.txt --no-progress
!uv pip install /teamspace/studios/this_studio/LoRA_Easy_Training_scripts_Backend/custom_scheduler/. --no-progress
%cd /teamspace/studios/this_studio/LoRA_Easy_Training_scripts_Backend/sd_scripts
!uv pip install -r requirements.txt
!mv /teamspace/studios/this_studio/LoRA_Easy_Training_scripts_Backend/notebook_scripts /teamspace/studios/this_studio/
!uv pip install torch torchvision torchaudio xformers
!uv pip install pytorch-optimizer==3.1.2
!uv pip uninstall rich
!rm -rf /teamspace/studios/this_studio/.cache
#clear_output()
display(HTML(f"<h1 style='color: yellow;'>Dependencias Instaladas, Reinicia el Kernel</h1>"))

# Elige tus parametros y espera unos segundos hasta que la carpeta se genere.

In [ ]:
#@title 🔧 Configuración rápida del entrenamiento
from notebook_scripts.training_config_ui import render_quick_training_config

render_quick_training_config(globals())


# Entrenamiento. Solo activa la celda y espera a que termine el entrenamiento.

In [ ]:
#@title 🚀 Entrenar LoRA
%cd /teamspace/studios/this_studio/
exec(open("notebook_scripts/run_training_cell.py").read(), globals())


## Opcional para no perder creditos

In [2]:
#elimina los modelos base antes de cerrar lightning para que no te cobren mas creditos por almacenamiento.
!rm -rf models

## Extras (WIP)

### 📚 Varias carpetas en el mismo conjunto de datos
A continuación se muestra una plantilla que le permite definir varias carpetas en su conjunto de datos. Debe incluir la ubicación de cada carpeta y puede establecer un número diferente de repeticiones para cada una. Para agregar más carpetas, simplemente copie y pegue las secciones que comienzan con `[[datasets.subsets]]`.

Al habilitar esto, se ignorará el número de repeticiones establecidas en la celda principal y también se ignorará la carpeta principal establecida por el nombre del proyecto.

Puede convertir uno de ellos en una carpeta de regularización agregando `is_reg = true`  
También puede establecer diferentes `keep_tokens`, `flip_aug`, etc.

In [ ]:
custom_dataset = """
[[datasets]]

[[datasets.subsets]]
image_dir = "/teamspace/studios/this_studio/lora_projects/example/dataset/good_images"
num_repeats = 3

[[datasets.subsets]]
image_dir = "/teamspace/studios/this_studio/lora_projects/example/dataset/normal_images"
num_repeats = 1

"""

In [ ]:
#@markdown ## Directorio base en Lightning.ai
from pathlib import Path
lightning_root = Path('/teamspace/studios/this_studio')
print(f"Trabajando en: {lightning_root}")


In [ ]:
custom_dataset = None

In [ ]:
##Calculador de Repeticiones ⌛📝
# Calcula el número de repeticiones a usar para entrenar tu lora, Recuerda que en `SDXL y Pony` se usa un batch de `4`.
# Si usas colab pro calcula tus repeticiones con `8` de batch size
# Define las Variables
# Número de imágenes
num_images = 24 # @param{type:"number"}
# Número de repeticiones
num_repeats = 2 # @param{type:"number"}
# Número de epocas
num_epochs = 30 # @param{type:"number"}
# Tamaño de lote
batch_size = 8 # @param{type:"number"}

# Calcula el resultado
resultado = (num_images * num_repeats * num_epochs) / batch_size

# Muestra el resultado
print("\33[96mEl total de repeticiones es:\033[0m", resultado)

In [ ]:
#@markdown ### 📂 Descomprimir conjunto de datos
#@markdown Sube un archivo `.zip` al almacenamiento de Lightning y descomprímelo en la carpeta deseada.
zip = "/teamspace/studios/this_studio/my_dataset.zip" #@param {type:"string"}
extract_to = "/teamspace/studios/this_studio/lora_projects/example/dataset" #@param {type:"string"}

import os, zipfile

os.makedirs(extract_to, exist_ok=True)
with zipfile.ZipFile(zip, 'r') as f:
  f.extractall(extract_to)

print("✅ Archivo extraído")
